##### Imports

In [ ]:
from scripts.validation import *
from scripts.plotting import * 

Progress tracking

In [2]:
# track progress
tqdm.pandas()

## OSFA moderation validation
- now that we have found toxicity-based clusters of comments, we want to validate the impact of moderations:
    - focusing on before moderation data against all moderation types
    - in terms of how well moderation strategies are able to reduce toxic behaviors
    - to do so, since the moderated comments are versions of the unmoderated comments
        - we look at how patterns change for each toxic profile
        - using the clusters found exlusively on unmoderated data

- we could do both global and local validations:
    - global, comparing full datasets against each other
    - local, comparing matching toxic profiles for each dataset

### Datasets

In [3]:
# simulated before mod
df_bef = pd.read_csv("../data_before_after/SIMULATOR_exante_bef_profiles.csv", encoding = "utf-8")
# moderated
df_ofsa = pd.read_csv("../data_before_after/SIMULATOR_exante_ofsa.csv", encoding = "utf-8")

##### Feature Engineering
- we want to derive features from the already extracted ones, could be more insightful

In [4]:
feat_engineering(df_ofsa)

100%|██████████| 2579/2579 [00:00<00:00, 151733.90it/s]


##### Feature selection
- keep only features shared across all datasets (real and simulated), to be able to do comparative analyses
- we exclude also the textual bodies and other irrelevant features for the analysis, such as parent_id and age (which has many missing values in the real dataset)

In [5]:
# features to keep
feats_shared = [col for col in df_bef.columns if col in df_ofsa.columns]

df_ofsa = df_ofsa[feats_shared].copy()

In [6]:
print(f"Shape of before moderation data: {df_bef.shape}")
df_bef.head(2)

Shape of before moderation data: (3131, 145)


,author,comment_id,std_body,gender,openness,conscientiousness,extraversion,agreeableness,neuroticism,num_punct,num_sents,num_words_upp,num_emoji,num_emoji_pos,num_emoji_neg,emoji_unique,emoji_list,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,polarity,subjectivity,valence,arousal,dominance,flesch,flesch_kincaid,fog,smog,ari,coleman_liau,dale_chall,linsear,difficult_words,num_words,num_words_unique,num_words_adj,num_words_noun,num_words_verb,num_words_lex,num_stopw,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,is_toxic,go,holy fuck,homosexual,say,know,well,bunch,shit,man,racist,would,stupid,also,guy,liberal,make,country,religion,real,crap,right,keep,people,dick,need,asshole,nothing,woman,bitch,one,give,fuck shit,die,black,immigrant,kill,dude,fucking,nigga,suck,take,good,try,idiot,call,insult insult,life,think,society,fuckin,someone,want,stop,see,f cking,way,time,damn,white,fuck,government,get,u,gay,even,death,thing,control,money,muslim,family,hope,really,like,is_real,cluster,sent_balance,sent_emoji_balance,punct_ratio,upper_ratio,emoji_ratio,adj_ratio,noun_ratio,verb_ratio,lex_ratio,stopw_ratio,words_sent_ratio,ttr,complex_ratio,tox_profile
0,joylukclub,2,since strongly lean towards republican side wh...,f,medium,very high,very low,low,very high,7,2,0,0,0,0,,,0.07,0.07,0.0,0.07,0.07,0.20,0.27,0.07,0.07,0.07,-0.052381,0.554762,0.205373,0.168915,0.189136,45.59,13.2,17.26,0.0,14.4,10.62,10.16,17.25,15.0,52,45,8,14,6,28,22,0.000711,0.000118,0.000166,0.000128,0.0,0.000160,0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.603497,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.797366,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,-0.07,0,0.134615,0.0,0.0,0.153846,0.269231,0.115385,0.538462,0.423077,26.000000,0.966667,0.288462,Healthy
1,WoodpeckerNo1,3,republican way go liberal policy destroy natio...,m,very low,very low,very low,very low,very high,12,7,0,0,0,0,,,0.11,0.11,0.0,0.11,0.00,0.16,0.26,0.11,0.00,0.05,0.052037,0.401667,0.168156,0.146662,0.168610,61.73,7.0,9.81,10.6,6.6,8.79,8.59,5.50,16.0,69,56,9,19,13,41,35,0.062836,0.000212,0.000633,0.000614,0.0,0.003106,0,0.26703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.593517,0.000000,0.0,0.361286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.245755,0.0,0.0,0.0,0.318712,0.0,0.0,0.0,0.241404,0.0,0.0,0.0,0.0,0.0,0.0,0.292843,0.227268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.297067,0.0,0.0,0.0,0.0,0.0,0.0,0,1,-0.10,0,0.173913,0.0,0.0,0.130435,0.275362,0.188406,0.594203,0.507246,9.857143,0.970588,0.231884,Offensive


In [7]:
print(f"Shape of moderated data: {df_ofsa.shape}")
df_ofsa.head(2)

Shape of moderated data: (2579, 144)


,author,comment_id,std_body,gender,openness,conscientiousness,extraversion,agreeableness,neuroticism,num_punct,num_sents,num_words_upp,num_emoji,num_emoji_pos,num_emoji_neg,emoji_unique,emoji_list,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,polarity,subjectivity,valence,arousal,dominance,flesch,flesch_kincaid,fog,smog,ari,coleman_liau,dale_chall,linsear,difficult_words,num_words,num_words_unique,num_words_adj,num_words_noun,num_words_verb,num_words_lex,num_stopw,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,is_toxic,go,holy fuck,homosexual,say,know,well,bunch,shit,man,racist,would,stupid,also,guy,liberal,make,country,religion,real,crap,right,keep,people,dick,need,asshole,nothing,woman,bitch,one,give,fuck shit,die,black,immigrant,kill,dude,fucking,nigga,suck,take,good,try,idiot,call,insult insult,life,think,society,fuckin,someone,want,stop,see,f cking,way,time,damn,white,fuck,government,get,u,gay,even,death,thing,control,money,muslim,family,hope,really,like,is_real,cluster,sent_balance,sent_emoji_balance,punct_ratio,upper_ratio,emoji_ratio,adj_ratio,noun_ratio,verb_ratio,lex_ratio,stopw_ratio,words_sent_ratio,ttr,complex_ratio
0,joylukclub,2,since strongly lean towards republican side wh...,f,medium,very high,very low,low,very high,7,2,0,0,0,0,,,0.07,0.07,0.0,0.07,0.07,0.20,0.27,0.07,0.07,0.07,-0.052381,0.554762,0.205373,0.168915,0.189136,45.59,13.2,17.26,0.0,14.4,10.62,10.16,17.25,15.0,52,45,8,14,6,28,22,0.000711,0.000118,0.000166,0.000128,0.0,0.000160,0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.615056,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.788483,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,2,-0.07,0,0.134615,0.0,0.0,0.153846,0.269231,0.115385,0.538462,0.423077,26.000000,0.966667,0.288462
1,WoodpeckerNo1,3,republican way go liberal policy destroy natio...,m,very low,very low,very low,very low,very high,12,7,0,0,0,0,,,0.11,0.11,0.0,0.11,0.00,0.16,0.26,0.11,0.00,0.05,0.052037,0.401667,0.168156,0.146662,0.168610,61.73,7.0,9.81,10.6,6.6,8.79,8.59,5.50,16.0,69,56,9,19,13,41,35,0.062836,0.000212,0.000633,0.000614,0.0,0.003106,0,0.26141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.618367,0.000000,0.0,0.352417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243774,0.0,0.0,0.0,0.30152,0.0,0.0,0.0,0.243548,0.0,0.0,0.0,0.0,0.0,0.0,0.284627,0.224816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.289618,0.0,0.0,0.0,0.0,0.0,0.0,0,1,-0.10,0,0.173913,0.0,0.0,0.130435,0.275362,0.188406,0.594203,0.507246,9.857143,0.970588,0.231884


##### Features for each dimension to evaluate

In [8]:
## toxic dimension
tox_feat = ["obscene", "threat", "insult", "identity_attack"]

## personality dimension
ocean_feat = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]

## sentiment/emotional dimension
sent_feat = ["fear", "anger", "anticip", "trust", "surprise", "sadness", "disgust", "joy", "valence", "arousal", "dominance", "subjectivity", "sent_balance", "sent_emoji_balance"]
# not considered:
# ("positive" - "negative") -> "sent_balance"
# "polarity", reduntant with "sent_balance"
# ("num_emoji_pos" - "num_emoji_neg") -> "sent_emoji_balance"

## linguistic dimension
ling_feat = ["num_words", "num_words_unique", "punct_ratio", "upper_ratio", "emoji_ratio", "adj_ratio", "noun_ratio", "verb_ratio", "lex_ratio", "stopw_ratio", "words_sent_ratio", "ttr"]
# not considered:
# count_features -> count_feature / "num_words"
# "emoji_unique"
# "emoji_list"

## readability dimension
read_feat = ["flesch", "flesch_kincaid", "fog", "smog", "ari", "coleman_liau", "dale_chall", "linsear", "complex_ratio"]
# not considered:
# "difficult_words" -> "complex_ratio" = "difficult_words"/"num_words"

## Global Validation

In [9]:
equal_feats, diff_feats, valid_metrics = validation_metrics_mod(df_bef, df_ofsa, tox_cols = tox_feat, pers_cols = ocean_feat,
                                                            sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                            pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "OSFA")

## Validating Unmoderated against OSFA data ##
GLOBAL VALIDATION
----------------------

Size of Unmoderated data: 3131
Size of OSFA data: 2579


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['obscene', 'threat', 'insult', 'identity_attack']
Significally different features (p-value < 0.01):
[]

Metrics for all significally different features:


## Local Validation

### Profiles mapping
- to validate how patterns change between before and after moderation, we don't map the clusters
- we instead consider the profiles found for the unmoderated data
- and annotate the moderated comments with their unmoderated toxic profile
- we therefore evaluate how each toxic profile has been affected by the specific moderation type

In [10]:
# retrieve original toxic profiles for the moderated comments
df_ofsa = df_ofsa.merge(df_bef[["comment_id", "tox_profile"]], on = "comment_id", how = "left")

In [11]:
df_ofsa.head(2)

,author,comment_id,std_body,gender,openness,conscientiousness,extraversion,agreeableness,neuroticism,num_punct,num_sents,num_words_upp,num_emoji,num_emoji_pos,num_emoji_neg,emoji_unique,emoji_list,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,polarity,subjectivity,valence,arousal,dominance,flesch,flesch_kincaid,fog,smog,ari,coleman_liau,dale_chall,linsear,difficult_words,num_words,num_words_unique,num_words_adj,num_words_noun,num_words_verb,num_words_lex,num_stopw,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,is_toxic,go,holy fuck,homosexual,say,know,well,bunch,shit,man,racist,would,stupid,also,guy,liberal,make,country,religion,real,crap,right,keep,people,dick,need,asshole,nothing,woman,bitch,one,give,fuck shit,die,black,immigrant,kill,dude,fucking,nigga,suck,take,good,try,idiot,call,insult insult,life,think,society,fuckin,someone,want,stop,see,f cking,way,time,damn,white,fuck,government,get,u,gay,even,death,thing,control,money,muslim,family,hope,really,like,is_real,cluster,sent_balance,sent_emoji_balance,punct_ratio,upper_ratio,emoji_ratio,adj_ratio,noun_ratio,verb_ratio,lex_ratio,stopw_ratio,words_sent_ratio,ttr,complex_ratio,tox_profile
0,joylukclub,2,since strongly lean towards republican side wh...,f,medium,very high,very low,low,very high,7,2,0,0,0,0,,,0.07,0.07,0.0,0.07,0.07,0.20,0.27,0.07,0.07,0.07,-0.052381,0.554762,0.205373,0.168915,0.189136,45.59,13.2,17.26,0.0,14.4,10.62,10.16,17.25,15.0,52,45,8,14,6,28,22,0.000711,0.000118,0.000166,0.000128,0.0,0.000160,0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.615056,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.788483,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,2,-0.07,0,0.134615,0.0,0.0,0.153846,0.269231,0.115385,0.538462,0.423077,26.000000,0.966667,0.288462,Healthy
1,WoodpeckerNo1,3,republican way go liberal policy destroy natio...,m,very low,very low,very low,very low,very high,12,7,0,0,0,0,,,0.11,0.11,0.0,0.11,0.00,0.16,0.26,0.11,0.00,0.05,0.052037,0.401667,0.168156,0.146662,0.168610,61.73,7.0,9.81,10.6,6.6,8.79,8.59,5.50,16.0,69,56,9,19,13,41,35,0.062836,0.000212,0.000633,0.000614,0.0,0.003106,0,0.26141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.618367,0.000000,0.0,0.352417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243774,0.0,0.0,0.0,0.30152,0.0,0.0,0.0,0.243548,0.0,0.0,0.0,0.0,0.0,0.0,0.284627,0.224816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.289618,0.0,0.0,0.0,0.0,0.0,0.0,0,1,-0.10,0,0.173913,0.0,0.0,0.130435,0.275362,0.188406,0.594203,0.507246,9.857143,0.970588,0.231884,Offensive


### Validation metrics

##### Healthy profile (no, we only consider toxic profiles)

In [12]:
#equal_neut, diff_neut, valid_metrics_neut = validation_metrics_mod(df_bef, df_ofsa, profile = "Healthy", tox_cols = tox_feat, pers_cols = ocean_feat, 
#                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
#                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "OSFA")

##### Vulgar profile

In [13]:
equal_vulg, diff_vulg, valid_metrics_vulg = validation_metrics_mod(df_bef, df_ofsa, profile = "Vulgar", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "OSFA")

## Validating Unmoderated against OSFA data ##
LOCAL VALIDATION: Vulgar profile
----------------------

Size of Unmoderated data: 356
Size of OSFA data: 287


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** threat **
	rel_mean_diff: -1.41714
	rel_std_diff: -8.68578
	rel_p10_diff: 0.46895
	rel_p25_diff: 0.63567
	rel_p50_diff: 0.33546
	rel_p75_diff: 0.14217
	rel_p90_diff: -0.02211
	KS: 0.23675
** identity_attack **
	rel_mean_diff: 0.06788
	rel_std_diff: -0.12463
	rel_p10_diff: 0.6825
	rel_p25_diff: 0.79235
	rel_p50_diff: 0.48518
	rel_p75_diff: 0.23042
	rel_p90_diff: 0.1502
	KS: 0.23394
** obscene **
	rel_mean_diff: 0.01384
	rel_std_diff: -0.22239
	rel_p10_diff: 0.81022
	rel_p25_diff: 0.91906
	rel_p50_diff: 0.90201
	rel_p75_diff: 0.10562
	rel_p90_diff: -0.13974
	KS: 0.28656


##### Offensive profile

In [14]:
equal_off, diff_off, valid_metrics_off = validation_metrics_mod(df_bef, df_ofsa, profile = "Offensive", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "OSFA")

## Validating Unmoderated against OSFA data ##
LOCAL VALIDATION: Offensive profile
----------------------

Size of Unmoderated data: 897
Size of OSFA data: 749


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** obscene **
	rel_mean_diff: -56.1851
	rel_std_diff: -33.31154
	rel_p10_diff: -0.02914
	rel_p25_diff: -0.04462
	rel_p50_diff: -0.42826
	rel_p75_diff: -1.75474
	rel_p90_diff: -32.2634
	KS: 0.20581
** identity_attack **
	rel_mean_diff: -16.39186
	rel_std_diff: -20.33739
	rel_p10_diff: 0.03843
	rel_p25_diff: 0.01313
	rel_p50_diff: -0.56172
	rel_p75_diff: -3.46735
	rel_p90_diff: -15.27815
	KS: 0.21909
** threat **
	rel_mean_diff: -6.4292
	rel_std_diff: -48.98596
	rel_p10_diff: -0.0053
	rel_p25_diff: -0.03011
	rel_p50_diff: -0.19489
	rel_p75_diff: -1.33677
	rel_p90_diff: -3.79485
	KS: 0.20515


##### Discriminatory profile

In [15]:
equal_disc, diff_disc, valid_metrics_disc = validation_metrics_mod(df_bef, df_ofsa, profile = "Discriminatory", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "OSFA")

## Validating Unmoderated against OSFA data ##
LOCAL VALIDATION: Discriminatory profile
----------------------

Size of Unmoderated data: 359
Size of OSFA data: 289


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** obscene **
	rel_mean_diff: -0.72474
	rel_std_diff: -0.65133
	rel_p10_diff: 0.59137
	rel_p25_diff: 0.73744
	rel_p50_diff: 0.58271
	rel_p75_diff: 0.12168
	rel_p90_diff: -1.49979
	KS: 0.24709
** identity_attack **
	rel_mean_diff: 0.2322
	rel_std_diff: 0.06848
	rel_p10_diff: 0.81431
	rel_p25_diff: 0.88357
	rel_p50_diff: 0.61615
	rel_p75_diff: 0.45946
	rel_p90_diff: 0.26428
	KS: 0.25055
** threat **
	rel_mean_diff: 0.16492
	rel_std_diff: -0.18105
	rel_p10_diff: 0.64809
	rel_p25_diff: 0.76358
	rel_p50_diff: 0.5462
	rel_p75_diff: 0.44674
	rel_p90_diff: 0.31111
	KS: 0.27349


##### Hostile profile

In [16]:
equal_host, diff_host, valid_metrics_host = validation_metrics_mod(df_bef, df_ofsa, profile = "Hostile", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "OSFA")

## Validating Unmoderated against OSFA data ##
LOCAL VALIDATION: Hostile profile
----------------------

Size of Unmoderated data: 242
Size of OSFA data: 190


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** threat **
	rel_mean_diff: 0.69487
	rel_std_diff: 0.45226
	rel_p10_diff: 0.90529
	rel_p25_diff: 0.85644
	rel_p50_diff: 0.80891
	rel_p75_diff: 0.76304
	rel_p90_diff: 0.83831
	KS: 0.46207
** obscene **
	rel_mean_diff: 0.58025
	rel_std_diff: -0.64025
	rel_p10_diff: 0.99962
	rel_p25_diff: 0.99903
	rel_p50_diff: 0.9661
	rel_p75_diff: 0.22907
	rel_p90_diff: 0.06708
	KS: 0.54102
** identity_attack **
	rel_mean_diff: 0.57881
	rel_std_diff: 0.25395
	rel_p10_diff: 0.97576
	rel_p25_diff: 0.9592
	rel_p50_diff: 0.89986
	rel_p75_diff: 0.81225
	rel_p90_diff: 0.53491
	KS: 0.38773
